In [42]:
import torch
from torch.utils.data import DataLoader, TensorDataset

import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

import json
import random

from data_loading_code import getData, processUserInput
from model import NeuralNet, LitNet

In [43]:
# get the preprocessed Data
x_train, y_train, x_val, y_val, vocabsize, word_vectorizer = getData("amazon_cells_labelled.txt")

Train data: torch.Size([900, 7277])
Train labels: torch.Size([900])
Validation data: torch.Size([100, 7277])
Validation labels: torch.Size([100])
Vocab size: 7277
Successfully preprocessed the data


In [44]:
# hyperparameters
EPOCHS = 150
BATCH_SIZE = 10
LEARNING_RATE = 0.001
INPUT_SIZE = vocabsize
HIDDEN_SIZE = 8
NUM_CLASSES = 2 # we only have two classes

In [45]:
# put data into Dataloaders
train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=BATCH_SIZE, shuffle=True, num_workers=7, persistent_workers=True)
val_loader = DataLoader(TensorDataset(x_val, y_val), batch_size=BATCH_SIZE, shuffle=False, num_workers=7, persistent_workers=True)

In [46]:
# initialise neural Net
model = NeuralNet(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES)
litModel = LitNet(model)

# initialise logger
logger = TensorBoardLogger("tb_logs", name="chatBotModel")

# intialise Trainer
trainer = L.Trainer(max_epochs=EPOCHS, 
                    callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
                    logger=logger,
                    fast_dev_run=False)

# train the model
trainer.fit(litModel, train_dataloaders=train_loader, val_dataloaders=val_loader)
trainer.save_checkpoint("best_model.ckpt")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params
------------------------------------
0 | model | NeuralNet | 58.3 K
------------------------------------
58.3 K    Trainable params
0         Non-trainable params
58.3 K    Total params
0.233     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [47]:
# load model
model = NeuralNet(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES)
litModel = LitNet.load_from_checkpoint("best_model.ckpt", model=model)

# disable randomness, dropout, etc...
model.eval()

NeuralNet(
  (l1): Linear(in_features=7277, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=2, bias=True)
  (relu): ReLU()
)

In [48]:
with open('responses.json', 'r') as json_data:
    responses = json.load(json_data)


In [49]:
# chatBot

# load response data for chatBot

bot_name = "Bot"
print("Let's chat! (type 'quit' to exit)")
while True:
    
    sentence = input("You: ")
    if sentence == "quit":
        break
    
    # process and predict for the user input
    userInput = processUserInput(sentence, word_vectorizer)
    output = model(userInput)
    
    _,pred = torch.max(output,dim=1)
    pred = pred.item()

    # select random entry from list for the prediction
    resp = random.choice(responses["responses"][pred]["answers"])
    print(f"{bot_name}: {resp}")

Let's chat! (type 'quit' to exit)
Bot: We're so glad to hear that you enjoyed our product! Your satisfaction is our top priority.
Bot: Thank you for bringing this to our attention. We take your feedback seriously and will use it to improve.
Bot: We apologize for any inconvenience you may have experienced. Your feedback helps us improve.
Bot: We're so glad to hear that you enjoyed our product! Your satisfaction is our top priority.
Bot: Thank you for bringing this to our attention. We take your feedback seriously and will use it to improve.
Bot: Thank you for bringing this to our attention. We take your feedback seriously and will use it to improve.
Bot: Thank you for bringing this to our attention. We take your feedback seriously and will use it to improve.
Bot: We're disappointed to hear that you weren't satisfied with our product/service. We'll work to make it right.
Bot: We're sorry to hear about your experience. Please reach out to us directly so we can address your concerns.
Bot: 